In [2]:
import sys
sys.path.append("lib/")
sys.path.append("../algorithms-project-01/collective")
import random
import time
import csv
import math
import imp
import Sorts
from Sorts import dual_pivot
from Sorts import introsort
import quicksort
from sort_inspector import Sequence

import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

This function generates tuples with lists to sort, and info about each as tuples:

In [3]:
def trial_gen(sizes,types=['shuffle']):
    types = list(types)
    for n in sizes:
        for typ in types:
            if typ == 'sorted':
                yield (typ,n,list(range(0,n)))
            elif typ == 'reverse':
                yield (typ,n,list(range(n-1,-1,-1)))
            elif typ == 'shuffle':
                t = list(range(0,n))
                random.shuffle(t)
                yield (typ,n,t)
            else: #typ is integer number of options
                t = [ random.randrange(0,typ) for x in range(0,n)]
                yield ('pick',len(set(t)),t)

Generates logarithmically-spaced sequences:

In [4]:
def log_range(start,end,jump = math.sqrt(math.e)):
    x = float(start)
    while x < end:
        yield int(x)
        x *= jump

Given a sort function, a sequence of tuples as created by `trial_gen()`, and a set of "modes" (which consist of additional output columns, and additional arguments to the sort function, generates a sequence of output rows including timings and swap/compare counts.

In [5]:
def run_tests(sortfn,trials,modes=iter([[],([],[],{})])):
    mode_headers = next(modes)
    modes = list(modes)
    yield( ['len','type','n_distinct','time','swaps','comparisons'] + mode_headers )
    for (typ,n_distinct,vals) in trials:
        for (mode_info,mode_args,mode_kwargs) in modes:
            randstate = random.getstate()
            T0 = time.process_time()
            sortfn(vals[:],*mode_args,**mode_kwargs)
            T1 = time.process_time()
            elapsed = T1 - T0
            random.setstate(randstate)
            mons = Sequence(vals)
            sortfn(mons,*mode_args,**mode_kwargs)
            yield( [len(vals),typ,n_distinct,elapsed,mons.count_swaps(),mons.count_comparisons()] + mode_info )

Modes for dual-pivot sort:

In [10]:
def dualp_modes(thresholds,subsorts=[('insertion',Sorts.insertion)],randomize=True):
    subsorts = list(subsorts)
    rands = list(randomize)
    yield(['randomize','threshold','subsort'])
    for threshold in thresholds:
        for (subsort_name,subsort) in subsorts:
            for randomize in rands:
                yield ([randomize,threshold,subsort_name],[],
                        {'randomize':randomize, 'threshold': threshold, 'subsort': subsort})

In [7]:
#list(run_tests(dual_pivot,trial_gen(log_range(2,5,2),['shuffle',2]),modes = dualp_modes(log_range(2,5,2))))

In [8]:
def write_results(out,results):
    with open(out,'w',newline ='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        headers = next(results)
        writer.writerow(headers)
        for row in results:
            writer.writerow(row)
            f.flush

In [25]:
write_results('./dual_pivot.csv',
              results = run_tests(dual_pivot, trial_gen(log_range(10,50000),
                                                        ['sorted','reverse','shuffle',2,5,10,100]),
                                  modes = dualp_modes(log_range(2,33,2),randomize=[True,False])
                                )
            )

In [14]:
write_results('./dual_pivot_subsort_cmp.csv',
              results = run_tests(dual_pivot, trial_gen(log_range(10,50000),
                                                        ['shuffle',10]),
                                  modes = dualp_modes(log_range(2,33,2),randomize=[True],
                                                      subsorts = [('insertion',Sorts.insertion),
                                                                  ('quicksort',quicksort.quicksort)]
                                                     )
                                )
            )

In [29]:
write_results('./introsort.csv',
               results = run_tests(introsort, trial_gen(log_range(10,50000),
                                                        ['sorted','reverse','shuffle',2,5,10,100]
                                                       ))
             )